<a href="https://colab.research.google.com/github/squirelt/UWB_ML/blob/main/UWB_ML_CNN_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UWB Machine Learning

This Notebook is dedicated for tarining uf nerual network Model for detection in Human Presence Sensor Project of ZHAW 


## Prepare Environment

In [1]:

# Check if NVIDIA GPU is enabled
!nvidia-smi



Wed Feb 16 22:19:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!python --version


Python 3.7.12


**Connect and authorize google drive with google colab:**

In [12]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls



MessageError: ignored

In [20]:
!ls

sample_data


**Open the UWB working directory on google drive**

---



In [21]:
#%cd /content/gdrive/MyDrive/Colab\ Notebooks/UWB_ML
!ls -a


.  ..  .config	sample_data


**Install required libraries**

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import torch 


### Define Functions

In [4]:
def weighted_cross_entropy(y_true, y_pred):
  return tf.nn.weighted_cross_entropy_with_logits(y_true, y_pred, 224.0)


## Create NN Model

- Convolutional Layers
- GRU Layers

In [46]:
length_of_CIR = 256
nr_of_samples = 2000

input_tensor = keras.Input(shape=(nr_of_samples, length_of_CIR, 1))

# create all conv layers

conv_1 = layers.Conv2D(4,5,activation='relu', padding='same', input_shape=input_tensor.shape)(input_tensor)
conv_2 = layers.Conv2D(4,5,strides=(1, 2),activation='relu', padding='same', input_shape=conv_1.shape)(conv_1)
conv_3 = layers.Conv2D(4,5,strides=(1, 2),activation='relu', padding='same', input_shape=conv_2.shape)(conv_2)
conv_4 = layers.Conv2D(4,5,strides=(1, 2),activation='relu', padding='same', input_shape=conv_2.shape)(conv_3)

conv_re = layers.Reshape((conv_4.shape[1],conv_4.shape[2]*conv_4.shape[3]))(conv_4)


gru_1 = layers.GRU(512,return_sequences=True)(conv_re)
gru_2 = layers.GRU(64, return_sequences=True)(gru_1)
dense_1 = layers.Dense(64, activation='linear')(gru_2)




print(conv_1.shape)
print(conv_2.shape)
print(conv_3.shape)
print(conv_4.shape)
print(conv_re.shape)
#print(final_reshape.shape)


model = keras.Model(inputs=input_tensor, outputs=dense_1)
model.summary()



(None, 2000, 256, 4)
(None, 2000, 128, 4)
(None, 2000, 64, 4)
(None, 2000, 32, 4)
(None, 2000, 128)
Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 2000, 256, 1)]    0         
                                                                 
 conv2d_44 (Conv2D)          (None, 2000, 256, 4)      104       
                                                                 
 conv2d_45 (Conv2D)          (None, 2000, 128, 4)      404       
                                                                 
 conv2d_46 (Conv2D)          (None, 2000, 64, 4)       404       
                                                                 
 conv2d_47 (Conv2D)          (None, 2000, 32, 4)       404       
                                                                 
 reshape_10 (Reshape)        (None, 2000, 128)         0         
                         

In [50]:
# compile model using the right loss function
# Use binary crossentropy loss
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              loss=weighted_cross_entropy,
              metrics=['accuracy'])

### Loda Model

In [29]:
# load model
model = keras.models.load_model('51-0.32.hdf5',compile=False)


## Load Training Data

In [7]:
torch.hub.download_url_to_file('https://drive.switch.ch/index.php/s/se1N8xiVaxC1S4t/download', 'tmp.zip')
!sudo unzip -q tmp.zip -d ./ && rm tmp.zip
!ls -a

  0%|          | 0.00/2.60G [00:00<?, ?B/s]

.  ..  .config	data3  sample_data


In [8]:

npz3 = np.load('./data3/train_input.npz')
npz4 = np.load('./data3/train_output.npz')

train_input = npz3['arr_0']
train_output = npz4['arr_0']

print(train_input.shape)
print(train_output.shape)


(4095, 2000, 256)
(4095, 2000, 64)


In [9]:
# load test data
npz1 = np.load('./data3/test_input.npz')
npz2 = np.load('./data3/test_output.npz')

test_input = npz1['arr_0']
test_output = npz2['arr_0']

print(test_input.shape)
print(test_output.shape)




(1024, 2000, 256)
(1024, 2000, 64)


## Training 

In [10]:
# define callback
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='{epoch:02d}-{val_loss:.2f}.hdf5',
    save_weights_only=False,
    save_freq='epoch',
    save_best_only=False)


In [11]:
print(train_input.shape)
print(test_output.shape)


(4095, 2000, 256)
(1024, 2000, 64)


In [51]:


#model.fit(train_input[0:5,:,:], train_output[0:5,:,:], epochs=1000, batch_size=2,validation_data=(test_input[0:5,:,:], test_output[0:128,:,:]),callbacks=[model_checkpoint_callback])

model.fit(train_input[0:5,:,:], train_output[0:5,:,:], epochs=500, batch_size=1) # try overfitting

Epoch 1/500
5/5 [==============================] - 3s 136ms/step - loss: 0.1199 - accuracy: 0.0127
Epoch 2/500
5/5 [==============================] - 1s 134ms/step - loss: 0.1029 - accuracy: 0.0145
Epoch 3/500
5/5 [==============================] - 1s 136ms/step - loss: 0.0997 - accuracy: 0.0124
Epoch 4/500
5/5 [==============================] - 1s 136ms/step - loss: 0.0962 - accuracy: 0.0113
Epoch 5/500
5/5 [==============================] - 1s 134ms/step - loss: 0.0930 - accuracy: 0.0102
Epoch 6/500
5/5 [==============================] - 1s 136ms/step - loss: 0.0908 - accuracy: 0.0107
Epoch 7/500
5/5 [==============================] - 1s 136ms/step - loss: 0.0887 - accuracy: 0.0093
Epoch 8/500
5/5 [==============================] - 1s 135ms/step - loss: 0.0878 - accuracy: 0.0092
Epoch 9/500
5/5 [==============================] - 1s 136ms/step - loss: 0.0863 - accuracy: 0.0092
Epoch 10/500
5/5 [==============================] - 1s 136ms/step - loss: 0.0849 - accuracy: 0.0115
Epoch 11/

In [52]:
import matplotlib.pyplot as plt

%matplotlib notebook

In [53]:
X_pred = model.predict(test_input[0:5,:,:])
x_pred_sigmoid = tf.nn.sigmoid(X_pred)
print(np.max(x_pred_sigmoid))


0.99997866


In [54]:
idx = 0
fig = plt.figure()
fig.add_subplot(1,2,1)
plt.imshow(test_output[0,:,:])
fig.add_subplot(1,2,2)
plt.imshow(x_pred_sigmoid[0,:,:])
plt.show()

<IPython.core.display.Javascript object>

In [55]:
# get precision
m = tf.keras.metrics.Precision()
m.update_state(test_output[0:5,:,:], x_pred_sigmoid[0:5,:,:])
m.result().numpy()


0.23011105

## **Inference**